In [25]:
import pandas as pd
import re
from difflib import SequenceMatcher
import numpy as np
from openpyxl import Workbook
from openpyxl.styles import PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows

# --- IMPROVED Serial extraction with better filtering ---
def extract_serials(text):
    """Extract valid serials: alphanumeric strings with at least one digit, 4+ characters"""
    if pd.isna(text):
        return []
    
    # Clean text by removing problematic characters but keeping separators
    cleaned_text = re.sub(r'[/.,;\'"\[\]\-\+\s]+', ' ', str(text).upper())
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    
    # Extract potential serials
    candidates = re.findall(r'[A-Z0-9]{4,}', cleaned_text)
    
    # Filter to only include those with at least one digit
    return [s for s in candidates if any(c.isdigit() for c in s)]

def normalize_serial(serial):
    """Remove separators and special characters for better matching"""
    if not serial:
        return ""
    return re.sub(r'[/.,;\'"\[\]\-\+\s]', '', str(serial).upper())

def similarity(a, b):
    """Enhanced similarity that normalizes serials before comparison"""
    norm_a = normalize_serial(a)
    norm_b = normalize_serial(b)
    
    # If normalized versions are identical, perfect match
    if norm_a == norm_b:
        return 1.0
    
    # Otherwise use sequence matcher
    return SequenceMatcher(None, norm_a, norm_b).ratio()

def find_closest_serial_match(search_serials, source_serials, threshold=0.85):
    """Find the closest matching serial from source_serials"""
    best_match = None
    best_score = 0
    for search_serial in search_serials:
        for source_serial in source_serials:
            score = similarity(search_serial, source_serial)
            if score > best_score and score >= threshold:
                best_score = score
                best_match = source_serial
    return best_match, best_score

def save_with_highlighting(df, output_file):
    """Save Excel file with yellow highlighting for matched rows"""
    wb = Workbook()
    ws = wb.active
    ws.title = "Matched Items"
    
    # Add dataframe to worksheet
    for r in dataframe_to_rows(df, index=False, header=True):
        ws.append(r)
    
    # Define yellow fill
    yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
    
    # Apply yellow highlighting to rows with matches
    for row_idx, (_, row) in enumerate(df.iterrows(), start=2):  # start=2 because row 1 is header
        # Check if this row has a match (non-empty Macro or UNITCOST)
        if str(row["Macro"]).strip() != "" or str(row["UNITCOST"]).strip() != "":
            # Highlight the entire row
            for col_idx in range(1, len(df.columns) + 1):
                cell = ws.cell(row=row_idx, column=col_idx)
                cell.fill = yellow_fill
    
    # Save the workbook
    wb.save(output_file)

# --- Load files ---
list_file = "List_Of_Items.csv"
source_file = "Source.csv"

df = pd.read_csv(list_file, encoding="ISO-8859-1")
source_df = pd.read_csv(source_file, encoding="ISO-8859-1")

# Clean column names
df.columns = df.columns.str.strip()
source_df.columns = source_df.columns.str.strip()

# Get relevant columns
actual_part_number_col = next((col for col in source_df.columns if 'Part Number' in col), None)
actual_unit_cost_col = next((col for col in source_df.columns if 'Unit Cost' in col), None)
item_col = next((col for col in df.columns if "Item" in col), None)

if not actual_part_number_col or not actual_unit_cost_col or not item_col:
    raise Exception("❌ Missing required column(s) in your input files.")

# Extract rows 250–500
subset_df = df.iloc[248:500].copy()

# Clean up text fields with improved cleaning
subset_df["Description"] = subset_df["Description"].fillna("").astype(str).str.upper().str.strip()
subset_df["Details"] = subset_df["Details"].fillna("").astype(str).str.upper().str.strip()
source_df[actual_part_number_col] = source_df[actual_part_number_col].fillna("").astype(str).str.upper().str.strip()

# Extract serial numbers from source
source_df["SERIALS"] = source_df[actual_part_number_col].apply(lambda x: extract_serials(x))
all_source_serials = set([s for sub in source_df["SERIALS"] for s in sub])

# Detect duplicates
full_item_list = df[item_col].dropna().astype(str)
duplicates_dict = {}
for item in full_item_list:
    matches = full_item_list[full_item_list == item]
    if len(matches) > 1:
        for idx in matches.index:
            duplicates_dict[idx] = item

# Initialize output columns - empty instead of "NA"
subset_df["Vendor"] = ""
subset_df["UNITCOST"] = ""
subset_df["DUPLICATES"] = ""
subset_df["Macro"] = ""

matches_found = 0

# --- Main loop: Match rows ---
for idx, row in subset_df.iterrows():
    combined_text = (row["Description"] + " " + row["Details"]).strip()
    search_serials = extract_serials(combined_text)
    
    if not search_serials:
        continue

    best_serial, score = find_closest_serial_match(search_serials, all_source_serials, threshold=0.75)
    
    if best_serial:
        matched_rows = source_df[source_df["SERIALS"].apply(lambda lst: best_serial in lst)]
        costs = matched_rows[actual_unit_cost_col].dropna().astype(float)

        if not costs.empty:
            if len(costs) == 1:
                chosen_cost = round(costs.iloc[0], 2)
            elif len(costs) == 2:
                chosen_cost = round(max(costs), 2)
            else:
                chosen_cost = round(np.median(costs), 2)

            subset_df.at[idx, "Macro"] = best_serial
            subset_df.at[idx, "UNITCOST"] = chosen_cost
            matches_found += 1

    if idx in duplicates_dict:
        subset_df.at[idx, "DUPLICATES"] = duplicates_dict[idx]

# --- Save results with highlighting ---
output_file = "Updated_List_of_Items_250_to_500.xlsx"
save_with_highlighting(subset_df, output_file)

# --- Summary ---
print(f"✅ Matching complete! Saved to '{output_file}'")
print(f"📊 Matches found: {matches_found} out of {len(subset_df)} rows")
print(f"🟡 Matched rows are highlighted in yellow")

✅ Matching complete! Saved to 'Updated_List_of_Items_250_to_500.xlsx'
📊 Matches found: 62 out of 252 rows
🟡 Matched rows are highlighted in yellow
